<a href="https://colab.research.google.com/github/arunmadhusud/Yolov8_fast-TensorRT-/blob/main/Tensorrt_onnx_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [ ]:
# Install Dependencies. Restart the current session after installing dependencies.
!pip install --upgrade --index-url https://pypi.ngc.nvidia.com nvidia-tensorrt
!pip install onnx
!pip install pycuda --user

In [2]:
# add TensorRT  and cuda library path to the environment variable
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/lib64
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/TensorRT-8.4.3.1/lib
!export PATH=$PATH:/usr/local/TensorRT-8.4.3.1/bin

In [ ]:
%pip install ultralytics

In [4]:
!pip install onnxscript

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 640.1/640.1 kB 16.8 MB/s eta 0:00:00


In [5]:
!pip install onnxruntime-gpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.1 MB/s eta 0:00:00


In [6]:
!pip install onnx onnxconverter-common

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 3.20.2 which is incompatible.


In [1]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.42 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.4/78.2 GB disk)


# PyTorch


In [2]:
import numpy as np
import cv2
import urllib.request
from PIL import Image
import torch
import torchvision.transforms as transforms
from ultralytics import YOLO
import torchvision

# Load the YOLO model
model = YOLO('/content/yolov8x.pt')

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

# Move the model to the GPU
model.model.to(device).eval()

torch.set_float32_matmul_precision("high")

# optimized_model = torch.compile(model.model,mode="reduce-overhead" )



100%|██████████| 131M/131M [00:00<00:00, 465MB/s] 


cuda


In [3]:
import torch
def nms_pytorch(P : torch.tensor ,scores,thresh_iou : float):
    """
    Apply non-maximum suppression to avoid detecting too many
    overlapping bounding boxes for a given object.
    Args:
        boxes: (tensor) The location preds for the image
            along with the class predscores, Shape: [num_boxes,5].
        thresh_iou: (float) The overlap thresh for suppressing unnecessary boxes.
    Returns:
        A list of filtered boxes, Shape: [ , 5]
    """

    # we extract coordinates for every
    # prediction box present in P
    x1 = P[:, 0]
    y1 = P[:, 1]
    x2 = P[:, 2]
    y2 = P[:, 3]

    # we extract the confidence scores as well
    # scores = P[:, 4]

    # calculate area of every block in P
    areas = (x2 - x1) * (y2 - y1)

    # sort the prediction boxes in P
    # according to their confidence scores
    order = scores.argsort()

    # initialise an empty list for
    # filtered prediction boxes
    keep = []


    while len(order) > 0:

        # extract the index of the
        # prediction with highest score
        # we call this prediction S
        idx = order[-1]

        # push S in filtered predictions list
        # keep.append(P[idx])
        keep.append(idx.item())

        # remove S from P
        order = order[:-1]

        # sanity check
        if len(order) == 0:
            break

        # select coordinates of BBoxes according to
        # the indices in order
        xx1 = torch.index_select(x1,dim = 0, index = order)
        xx2 = torch.index_select(x2,dim = 0, index = order)
        yy1 = torch.index_select(y1,dim = 0, index = order)
        yy2 = torch.index_select(y2,dim = 0, index = order)

        # find the coordinates of the intersection boxes
        xx1 = torch.max(xx1, x1[idx])
        yy1 = torch.max(yy1, y1[idx])
        xx2 = torch.min(xx2, x2[idx])
        yy2 = torch.min(yy2, y2[idx])

        # find height and width of the intersection boxes
        w = xx2 - xx1
        h = yy2 - yy1

        # take max with 0.0 to avoid negative w and h
        # due to non-overlapping boxes
        w = torch.clamp(w, min=0.0)
        h = torch.clamp(h, min=0.0)

        # find the intersection area
        inter = w*h

        # find the areas of BBoxes according the indices in order
        rem_areas = torch.index_select(areas, dim = 0, index = order)

        # find the union of every prediction T in P
        # with the prediction S
        # Note that areas[idx] represents area of S
        union = (rem_areas - inter) + areas[idx]

        # find the IoU of every prediction in P with S
        IoU = inter / union

        # keep the boxes with IoU less than thresh_iou
        mask = IoU < thresh_iou
        order = order[mask]

    return keep

In [4]:
import numpy as np
import cv2
from PIL import Image
import torch
import torchvision.transforms as transforms
# import onnxruntime as ort
from torchvision.ops import nms
import urllib.request
import os
from skimage import io
from skimage.transform import resize
import time

BATCH_SIZE = 1

class_names_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
                    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
                    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
                    'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
                    'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
                    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
                    'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
                    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
                    'scissors', 'teddy bear', 'hair drier', 'toothbrush']

def pytorch_run(model,image_array):
  """
  Inference using pytorch model
  """
  input_tensor = torch.from_numpy(image_array).transpose(1,3).transpose(2,3)
  input_tensor = input_tensor.to(device)
  results = model(input_tensor)
  return results


def draw_boxes(pytorch_results,image_array, video =False):
  """
  The raw output from yolo model contains information about bouding box coordinates and class probabilities.
  The first four indices corresponds to bounding box dimensions and the last dimension corresponds to class probabilities.
  This function will extract these information, retains bounding box with confidence score above 0.25 and perform an nms suppression to avoid detecting too many
  overlapping bounding boxes for a given object.The final output will be an image with bounding boxes drawn with clear labels to it.
  """

  # image_array = np.array(np.repeat(np.expand_dims(np.array(img), axis=0), BATCH_SIZE, axis=0)) * 255
  image_array = (image_array* 255).astype(np.uint8)

  save_dir = '/content/annotated_images'
  os.makedirs(save_dir, exist_ok=True)

  for batch_idx in range(pytorch_results.shape[0]):
      predictions = pytorch_results[batch_idx].permute(1, 0).unsqueeze(0)
      bounding_box_predictions = predictions[:, :, :4]
      class_probabilities = predictions[:, :, 4:]

      # Convert bounding box predictions to standard format
      x1 = bounding_box_predictions[:, :, 0] - bounding_box_predictions[:, :, 2] / 2
      y1 = bounding_box_predictions[:, :, 1] - bounding_box_predictions[:, :, 3] / 2
      x2 = bounding_box_predictions[:, :, 0] + bounding_box_predictions[:, :, 2] / 2
      y2 = bounding_box_predictions[:, :, 1] + bounding_box_predictions[:, :, 3] / 2
      bounding_boxes = torch.stack([x1, y1, x2, y2], dim=2)

      # Apply non-maximum suppression to bounding boxes
      confidence_scores = torch.max(class_probabilities, dim=2)[0]
      threshold = 0.25
      mask = confidence_scores >= threshold
      bounding_boxes = bounding_boxes[mask]
      class_probabilities = class_probabilities[mask]
      confidence_scores = confidence_scores[mask]

      # Apply non-maximum suppression to the bounding boxes
      nms_indices = nms_pytorch(bounding_boxes, confidence_scores, 0.45)
      selected_predictions = torch.cat([bounding_boxes[nms_indices], class_probabilities[nms_indices]], dim=1)

      # Extract bounding boxes and class labels for selected predictions
      selected_bounding_boxes = selected_predictions[:, :4]
      selected_class_probabilities = selected_predictions[:, 4:]
      selected_class_labels = torch.argmax(selected_class_probabilities, dim=1)

      # Annotate the image with bounding boxes and labels
      for i in range(len(selected_bounding_boxes)):
          x1, y1, x2, y2 = selected_bounding_boxes[i].int().tolist()
          label = class_names_list[selected_class_labels[i].item()]
          confidence = confidence_scores[nms_indices[i]].item()
          color = (255, 0, 0)  # Red color
          thickness = 2
          # image_array[batch_idx] = image_array[batch_idx]*255
          cv2.rectangle(image_array[batch_idx], (x1, y1), (x2, y2), color, thickness)
          cv2.putText(image_array[batch_idx], f'{label} {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, thickness)

      if not video:
        # Save the annotated image for the current batch
        save_path = os.path.join(save_dir, f'annotated_image_{batch_idx}.jpg')
        rgb_image = cv2.cvtColor(image_array[batch_idx], cv2.COLOR_BGR2RGB)
        cv2.imwrite(save_path, rgb_image)
  if video:
    return image_array

# Example usage using a sample image:
try:
    image_url = 'https://ultralytics.com/images/bus.jpg'
    image_path = '/content/bus.jpg'
    urllib.request.urlretrieve(image_url, image_path)
except Exception as e:
    print("Error downloading image:", e)


image_url = 'https://ultralytics.com/images/bus.jpg'
img = resize(io.imread(image_url), (640, 640))
image_array = np.array(np.repeat(np.expand_dims(np.array(img, dtype=np.float32), axis=0), BATCH_SIZE, axis=0), dtype=np.float32)
pytorch_results = pytorch_run(model.model, image_array)
draw_boxes(pytorch_results[0], image_array)



In [5]:
from google.colab import files

# Upload the video file
uploaded = files.upload()

Saving video1 (online-video-cutter.com).mp4 to video1 (online-video-cutter.com).mp4


In [6]:
# Get the path of the uploaded video file
video_path = next(iter(uploaded))

In [7]:

def process_video(model, input_video_path, output_video_path):
    """
    This function extracts the frames from uploaded video, detect the objects and draw bounding boxes with clear labels.
    The frames with the bounding box and detected labels are written back to an output video with fps equal
    corresponding to average processing time for each frome taken by the model
    """
    cap = cv2.VideoCapture(input_video_path)
    original_fps = cap.get(cv2.CAP_PROP_FPS)  # Get the FPS of the input video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, original_fps, (640, 640))  # Temporary FPS setting

    batch_frames = []
    BATCH_SIZE = 1 # Define your batch size
    frame_count = 0
    total_processing_time = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_resized = cv2.resize(frame, (640, 640))
        batch_frames.append(frame_resized)

        if len(batch_frames) == BATCH_SIZE:
            batch_frames = np.array(batch_frames, dtype=np.float32) / 255.0
            batch_tensor = batch_frames.astype(np.float32)

            start_time = time.time()
            pytorch_results = pytorch_run(model, batch_tensor)
            processing_time = time.time() - start_time
            total_processing_time += processing_time
            frame_count += len(batch_frames)

            annotated_frames = draw_boxes(pytorch_results[0], batch_frames, video=True)
            for annotated_frame in annotated_frames:
                fps_text = f'FPS: {frame_count / total_processing_time:.2f}'
                cv2.putText(annotated_frame, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                out.write(annotated_frame)

            batch_frames = []

    if batch_frames:  # Process remaining frames if any
        print("length of additional batch frames", len(batch_frames))
        last_frame = batch_frames[-1]
        while len(batch_frames) < BATCH_SIZE:
            batch_frames.append(last_frame)
        batch_frames = np.array(batch_frames, dtype=np.float32) / 255.0
        batch_tensor = batch_frames.astype(np.float32)

        start_time = time.time()
        pytorch_results = pytorch_run(model, batch_tensor)
        processing_time = time.time() - start_time
        total_processing_time += processing_time
        frame_count += len(batch_frames)

        annotated_frames = draw_boxes(pytorch_results[0], batch_frames, video=True)
        for annotated_frame in annotated_frames:
            fps_text = f'FPS: {frame_count / total_processing_time:.2f}'
            cv2.putText(annotated_frame, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            out.write(annotated_frame)

    cap.release()
    out.release()

    # Calculate the effective FPS
    effective_fps = frame_count / total_processing_time

    print("effective_fps",effective_fps )

    # Re-encode the video with the correct FPS
    temp_video_path = 'temp_output_video.mp4'
    cap = cv2.VideoCapture(output_video_path)
    out = cv2.VideoWriter(temp_video_path, fourcc, 60, (640, 640))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    cap.release()
    out.release()

    # Ensure the temporary file was created before renaming
    if os.path.exists(temp_video_path):
        os.remove(output_video_path)
        os.rename(temp_video_path, output_video_path)
    else:
        print("Error: temp_output_video.mp4 was not created successfully.")

# Set the ONNX model path
onnx_path = 'yolov8x.onnx'  # Path to your ONNX model

# Set the output video path
output_video_path = 'output_video.mp4'  # Path to save the output video

# Process the uploaded video
process_video(model.model, video_path, output_video_path)

# Download the output video
files.download(output_video_path)

37.850615588870355


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Onnx Runtime

In [8]:
import torch
from ultralytics import YOLO

# Load the YOLO model
model = YOLO('/content/yolov8x.pt')

# Create a dummy input tensor with the appropriate input size
dummy_input = torch.randn(1, 3, 640, 640)

# Ensure the model is on CPU for exporting
model.model.to('cpu')
model.model.eval()
model.model.float()
model.model = model.model.fuse()

# Export the model to ONNX
onnx_path = '/content/yolov8x.onnx'
torch.onnx.export(model.model, dummy_input, onnx_path, verbose=True, opset_version=17)

print(f"Model has been successfully exported to {onnx_path}")


YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


/usr/local/lib/python3.10/dist-packages/ultralytics/nn/modules/head.py:92: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.dynamic or self.shape != shape:
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/tal.py:299: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for i, stride in enumerate(strides):


Model has been successfully exported to /content/yolov8x.onnx


In [9]:
import onnx
onnx_model = onnx.load("/content/yolov8x.onnx")
onnx.checker.check_model(onnx_model)

In [10]:
import onnxruntime
onnxruntime.get_available_providers()

['TensorrtExecutionProvider',
 'CUDAExecutionProvider',
 'AzureExecutionProvider',
 'CPUExecutionProvider']

In [ ]:
# import onnx
# from onnxconverter_common import float16

# model = onnx.load("/content/yolov8x.onnx")
# model_fp16 = float16.convert_float_to_float16(model)
# onnx.save(model_fp16, "/content/yolov8x_fp16.onnx")

In [ ]:
# import onnx

# # Load the original ONNX model
# model = onnx.load("/content/yolov8x.onnx")

# # Inspect the model's input and output data types
# for input in model.graph.input:
#     print(f"Input: {input.name}, Type: {input.type}")

# for output in model.graph.output:
#     print(f"Output: {output.name}, Type: {output.type}")


In [11]:
import torch
def nms_pytorch(P : torch.tensor ,scores,thresh_iou : float):
    """
    Apply non-maximum suppression to avoid detecting too many
    overlapping bounding boxes for a given object.
    Args:
        boxes: (tensor) The location preds for the image
            along with the class predscores, Shape: [num_boxes,5].
        thresh_iou: (float) The overlap thresh for suppressing unnecessary boxes.
    Returns:
        A list of filtered boxes, Shape: [ , 5]
    """

    # we extract coordinates for every
    # prediction box present in P
    x1 = P[:, 0]
    y1 = P[:, 1]
    x2 = P[:, 2]
    y2 = P[:, 3]

    # we extract the confidence scores as well
    # scores = P[:, 4]

    # calculate area of every block in P
    areas = (x2 - x1) * (y2 - y1)

    # sort the prediction boxes in P
    # according to their confidence scores
    order = scores.argsort()

    # initialise an empty list for
    # filtered prediction boxes
    keep = []


    while len(order) > 0:

        # extract the index of the
        # prediction with highest score
        # we call this prediction S
        idx = order[-1]

        # push S in filtered predictions list
        # keep.append(P[idx])
        keep.append(idx.item())

        # remove S from P
        order = order[:-1]

        # sanity check
        if len(order) == 0:
            break

        # select coordinates of BBoxes according to
        # the indices in order
        xx1 = torch.index_select(x1,dim = 0, index = order)
        xx2 = torch.index_select(x2,dim = 0, index = order)
        yy1 = torch.index_select(y1,dim = 0, index = order)
        yy2 = torch.index_select(y2,dim = 0, index = order)

        # find the coordinates of the intersection boxes
        xx1 = torch.max(xx1, x1[idx])
        yy1 = torch.max(yy1, y1[idx])
        xx2 = torch.min(xx2, x2[idx])
        yy2 = torch.min(yy2, y2[idx])

        # find height and width of the intersection boxes
        w = xx2 - xx1
        h = yy2 - yy1

        # take max with 0.0 to avoid negative w and h
        # due to non-overlapping boxes
        w = torch.clamp(w, min=0.0)
        h = torch.clamp(h, min=0.0)

        # find the intersection area
        inter = w*h

        # find the areas of BBoxes according the indices in order
        rem_areas = torch.index_select(areas, dim = 0, index = order)

        # find the union of every prediction T in P
        # with the prediction S
        # Note that areas[idx] represents area of S
        union = (rem_areas - inter) + areas[idx]

        # find the IoU of every prediction in P with S
        IoU = inter / union

        # keep the boxes with IoU less than thresh_iou
        mask = IoU < thresh_iou
        order = order[mask]

    return keep

In [12]:
import numpy as np
import cv2
from PIL import Image
import torch
import torchvision.transforms as transforms
import onnxruntime as ort
from torchvision.ops import nms
import urllib.request
import os
from skimage import io
from skimage.transform import resize
import time

BATCH_SIZE = 1

class_names_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
                    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
                    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
                    'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
                    'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
                    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
                    'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
                    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
                    'scissors', 'teddy bear', 'hair drier', 'toothbrush']

def onnx_run(onnx_path,image_array,BATCH_SIZE=1):
  """
  Inference using Onnx runtime
  """

  providers = [("CUDAExecutionProvider", {"device_id": torch.cuda.current_device(),
                                          "user_compute_stream": str(torch.cuda.current_stream().cuda_stream)})]  # Use default CUDA device and stream

  sess_options = ort.SessionOptions()
  sess_options.log_severity_level = 0
  ort_session = ort.InferenceSession(onnx_path, sess_options=sess_options, providers=providers)


  # image_array = np.array(np.repeat(np.expand_dims(np.array(img, dtype=np.float32), axis=0), BATCH_SIZE, axis=0), dtype=np.float32)
  # image_array = np.array(img, dtype=np.float32)
  # print(image_array.shape)
  input_tensor = np.transpose(image_array, (0, 3, 1, 2))


  #IOBinding
  input_names = ort_session.get_inputs()[0].name
  input_shapes = ort_session.get_inputs()[0].shape
  input_type = ort_session.get_inputs()[0].type
  # print(input_names, input_shapes, input_type)

  output_names = ort_session.get_outputs()[0].name
  output_shapes = ort_session.get_outputs()[0].shape
  output_type = ort_session.get_outputs()[0].type
  # print(output_names, output_shapes, output_type)


  io_binding = ort_session.io_binding()
  io_binding.bind_cpu_input(input_names, input_tensor)
  io_binding.bind_output(output_names, 'cpu')

  start_time = time.time()
  ort_session.run_with_iobinding(io_binding)
  end_time = time.time()
  time_elapsed = end_time - start_time
  # print(time_elapsed)

  ort_outs = io_binding.copy_outputs_to_cpu()
  onnx_results = torch.tensor(ort_outs[0])

  return onnx_results


def draw_boxes(onnx_results,image_array, video =False):
  """
  The raw output from yolo model contains information about bouding box coordinates and class probabilities.
  The first four indices corresponds to bounding box dimensions and the last dimension corresponds to class probabilities.
  This function will extract these information, retains bounding box with confidence score above 0.25 and perform an nms suppression to avoid detecting too many
  overlapping bounding boxes for a given object.The final output will be an image with bounding boxes drawn with clear labels to it.
  """

  # image_array = np.array(np.repeat(np.expand_dims(np.array(img), axis=0), BATCH_SIZE, axis=0)) * 255
  image_array = (image_array* 255).astype(np.uint8)

  save_dir = '/content/annotated_images'
  os.makedirs(save_dir, exist_ok=True)

  for batch_idx in range(onnx_results.shape[0]):
      predictions = onnx_results[batch_idx].permute(1, 0).unsqueeze(0)
      bounding_box_predictions = predictions[:, :, :4]
      class_probabilities = predictions[:, :, 4:]

      # Convert bounding box predictions to standard format
      x1 = bounding_box_predictions[:, :, 0] - bounding_box_predictions[:, :, 2] / 2
      y1 = bounding_box_predictions[:, :, 1] - bounding_box_predictions[:, :, 3] / 2
      x2 = bounding_box_predictions[:, :, 0] + bounding_box_predictions[:, :, 2] / 2
      y2 = bounding_box_predictions[:, :, 1] + bounding_box_predictions[:, :, 3] / 2
      bounding_boxes = torch.stack([x1, y1, x2, y2], dim=2)

      # Apply non-maximum suppression to bounding boxes
      confidence_scores = torch.max(class_probabilities, dim=2)[0]
      threshold = 0.25
      mask = confidence_scores >= threshold
      bounding_boxes = bounding_boxes[mask]
      class_probabilities = class_probabilities[mask]
      confidence_scores = confidence_scores[mask]

      # Apply non-maximum suppression to the bounding boxes
      nms_indices = nms_pytorch(bounding_boxes, confidence_scores, 0.45)
      selected_predictions = torch.cat([bounding_boxes[nms_indices], class_probabilities[nms_indices]], dim=1)

      # Extract bounding boxes and class labels for selected predictions
      selected_bounding_boxes = selected_predictions[:, :4]
      selected_class_probabilities = selected_predictions[:, 4:]
      selected_class_labels = torch.argmax(selected_class_probabilities, dim=1)

      # Annotate the image with bounding boxes and labels
      for i in range(len(selected_bounding_boxes)):
          x1, y1, x2, y2 = selected_bounding_boxes[i].int().tolist()
          label = class_names_list[selected_class_labels[i].item()]
          confidence = confidence_scores[nms_indices[i]].item()
          color = (255, 0, 0)  # Red color
          thickness = 2
          # image_array[batch_idx] = image_array[batch_idx]*255
          cv2.rectangle(image_array[batch_idx], (x1, y1), (x2, y2), color, thickness)
          cv2.putText(image_array[batch_idx], f'{label} {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, thickness)

      if not video:
        # Save the annotated image for the current batch
        save_path = os.path.join(save_dir, f'annotated_image_{batch_idx}.jpg')
        rgb_image = cv2.cvtColor(image_array[batch_idx], cv2.COLOR_BGR2RGB)
        cv2.imwrite(save_path, rgb_image)
  if video:
    return image_array


try:
    image_url = 'https://ultralytics.com/images/bus.jpg'
    image_path = '/content/bus.jpg'
    urllib.request.urlretrieve(image_url, image_path)
except Exception as e:
    print("Error downloading image:", e)


image_url = 'https://ultralytics.com/images/bus.jpg'
img = resize(io.imread(image_url), (640, 640))
image_array = np.array(np.repeat(np.expand_dims(np.array(img, dtype=np.float32), axis=0), BATCH_SIZE, axis=0), dtype=np.float32)

onnx_path = '/content/yolov8x.onnx'
onnx_results = onnx_run(onnx_path, image_array)
draw_boxes(onnx_results, image_array)


In [13]:
from google.colab import files

# Upload the video file
uploaded = files.upload()

# Get the path of the uploaded video file
video_path = next(iter(uploaded))

Saving video1 (online-video-cutter.com)(1).mp4 to video1 (online-video-cutter.com)(1).mp4


In [14]:
def process_video(onnx_path, input_video_path, output_video_path):
    """
    This function extracts the frames from uploaded video, detect the objects and draw bounding boxes with clear labels.
    The frames with the bounding box and detected labels are written back to an output video with fps equal
    corresponding to average processing time for each frome taken by the model
    """
    cap = cv2.VideoCapture(input_video_path)
    original_fps = cap.get(cv2.CAP_PROP_FPS)  # Get the FPS of the input video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, original_fps, (640, 640))  # Temporary FPS setting

    batch_frames = []
    BATCH_SIZE = 1 # Define your batch size
    frame_count = 0
    total_processing_time = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_resized = cv2.resize(frame, (640, 640))
        batch_frames.append(frame_resized)

        if len(batch_frames) == BATCH_SIZE:
            batch_frames = np.array(batch_frames, dtype=np.float32) / 255.0
            batch_tensor = batch_frames.astype(np.float32)

            start_time = time.time()
            # pytorch_results = pytorch_run(model, batch_tensor)
            onnx_results = onnx_run(onnx_path, batch_tensor)
            processing_time = time.time() - start_time

            total_processing_time += processing_time
            frame_count += BATCH_SIZE

            # annotated_frames = draw_boxes(pytorch_results[0], batch_frames, video=True)
            annotated_frames = draw_boxes(onnx_results, batch_frames, video=True)
            for annotated_frame in annotated_frames:
                fps_text = f'FPS: {frame_count / total_processing_time:.2f}'
                cv2.putText(annotated_frame, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                out.write(annotated_frame)

            batch_frames = []

    if batch_frames:  # Process remaining frames if any
        last_frame = batch_frames[-1]
        while len(batch_frames) < BATCH_SIZE:
            batch_frames.append(last_frame)
        batch_frames = np.array(batch_frames, dtype=np.float32) / 255.0
        batch_tensor = batch_frames.astype(np.float32)

        start_time = time.time()
        # pytorch_results = pytorch_run(model, batch_tensor)
        onnx_results = onnx_run(onnx_path, batch_tensor)
        processing_time = time.time() - start_time

        total_processing_time += processing_time
        frame_count += len(batch_frames)

        # annotated_frames = draw_boxes(pytorch_results[0], batch_frames, video=True)
        annotated_frames = draw_boxes(onnx_results, batch_frames, video=True)
        for annotated_frame in annotated_frames:
            fps_text = f'FPS: {frame_count / total_processing_time:.2f}'
            cv2.putText(annotated_frame, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            out.write(annotated_frame)

    cap.release()
    out.release()

    # Calculate the effective FPS
    effective_fps = frame_count / total_processing_time

    print(effective_fps )

    # Re-encode the video with the correct FPS
    temp_video_path = 'temp_output_video.mp4'
    cap = cv2.VideoCapture(output_video_path)
    out = cv2.VideoWriter(temp_video_path, fourcc, 60, (640, 640))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    cap.release()
    out.release()

    # Ensure the temporary file was created before renaming
    if os.path.exists(temp_video_path):
        os.remove(output_video_path)
        os.rename(temp_video_path, output_video_path)
    else:
        print("Error: temp_output_video.mp4 was not created successfully.")

# Set the ONNX model path
onnx_path = 'yolov8x.onnx'  # Path to your ONNX model

# Set the output video path
output_video_path = 'output_video.mp4'  # Path to save the output video

# Process the uploaded video
process_video(onnx_path, video_path, output_video_path)

# Download the output video
files.download(output_video_path)

0.27098718280516293


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# TensorRT


In [15]:
from ultralytics import YOLO

model = YOLO("yolov8x.pt")
model.fuse()
model.info(verbose=False)
model.export(format='onnx')

YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs
Ultralytics YOLOv8.2.42 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.00GHz)

PyTorch: starting from 'yolov8x.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (130.5 MB)

ONNX: starting export with onnx 1.16.1 opset 17...
ONNX: export success ✅ 10.4s, saved as 'yolov8x.onnx' (260.4 MB)

Export complete (18.8s)
Results saved to /content
Predict:         yolo predict task=detect model=yolov8x.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8x.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8x.onnx'

In [16]:
import tensorrt as trt
import numpy as np
import torch


# set the flag to True to run inference in FP16
USE_FP16 = True
np_target_dtype = np.float32
target_dtype = torch.float32


In [18]:
onnx_filename = "/content/yolov8x.onnx"
BATCH_SIZE = 1

# Create a logger object with a severity level of INFO.
logger = trt.Logger(trt.Logger.INFO)

# create a Builder object. This object will be used to build the TensorRT engine from the network definition
builder = trt.Builder(logger)

#Network will be created with explicit batch size. This means that the batch size of the input tensors-
#will be specified explicitly during the engine building process, rather than being determined automatically by TensorRT.
network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))

# Create a parser object that can be used to parse an ONNX model file and-
# convert it to a TensorRT network object.
parser = trt.OnnxParser(network, logger)

# Parse the ONNX model to the TensorRT network
with open(onnx_filename, 'rb') as model_file:
    if not parser.parse(model_file.read()):
        for error in range(parser.num_errors):
            print(parser.get_error(error))

# Create an optimization profile object that can be used to optimize the TensorRT engine for a specific range of input shapes.
profile = builder.create_optimization_profile()

#Set the minimum/optimum/maximum dimensions for a dynamic input tensor in the optimization profile object.
profile.set_shape('input', (BATCH_SIZE, 3, 640, 640), (BATCH_SIZE, 3, 640, 640), (BATCH_SIZE, 3, 640, 640))

#Create a builder configuration object that can be used to configure the TensorRT engine builder
config = builder.create_builder_config()

# Set the precision to float16 if USE_FP16 is True
if USE_FP16:
    config.set_flag(trt.BuilderFlag.FP16)
    config.set_flag(trt.BuilderFlag.PREFER_PRECISION_CONSTRAINTS)

config.add_optimization_profile(profile)

max_workspace_size = 1 << 30  # 1GB
config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, max_workspace_size)


# Build the TensorRT engine from the optimized network [DEPRECATED METHOD]
# engine = builder.build_engine(network, config)

# Build the serialized TensorRT network blob
trt_blob = builder.build_serialized_network(network, config)

# Create TensorRT runtime object
runtime = trt.Runtime(trt.Logger(trt.Logger.WARNING))

# Clean up parser if no longer needed
del parser

# Deserialize CUDA engine from the serialized blob
engine = runtime.deserialize_cuda_engine(trt_blob)



In [19]:
# Save the TensorRT engine to a file

# [FOR DEPRECRATED METHOD]
# with open('model.trt', 'wb') as engine_file:
#     engine_file.write(engine.serialize())

# For deserialize_cuda_engine method
# Write the serialized blob to a file
with open('model.trt', 'wb') as f:
    f.write(trt_blob)

print(f"Serialized TensorRT engine saved to {'model.trt'}")

Serialized TensorRT engine saved to model.trt


In [24]:
# load the TensorRT engine from the saved file
%%time

import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

# Create TensorRT runtime object
runtime = trt.Runtime(trt.Logger(trt.Logger.WARNING))

# Load the serialized engine from file
with open('model.trt', 'rb') as f:
    trt_blob = f.read()
    engine = runtime.deserialize_cuda_engine(trt_blob)

print(f"Serialized TensorRT engine loaded from {'model.trt'}")

Serialized TensorRT engine loaded from model.trt
CPU times: user 366 ms, sys: 244 ms, total: 610 ms
Wall time: 632 ms


In [25]:
import torch
import time
def nms_pytorch(P : torch.tensor ,scores,thresh_iou : float):
    """
    Apply non-maximum suppression to avoid detecting too many
    overlapping bounding boxes for a given object.
    Args:
        boxes: (tensor) The location preds for the image
            along with the class predscores, Shape: [num_boxes,5].
        thresh_iou: (float) The overlap thresh for suppressing unnecessary boxes.
    Returns:
        A list of filtered boxes, Shape: [ , 5]
    """

    # we extract coordinates for every
    # prediction box present in P
    x1 = P[:, 0]
    y1 = P[:, 1]
    x2 = P[:, 2]
    y2 = P[:, 3]

    # we extract the confidence scores as well
    # scores = P[:, 4]

    # calculate area of every block in P
    areas = (x2 - x1) * (y2 - y1)

    # sort the prediction boxes in P
    # according to their confidence scores
    order = scores.argsort()

    # initialise an empty list for
    # filtered prediction boxes
    keep = []


    while len(order) > 0:

        # extract the index of the
        # prediction with highest score
        # we call this prediction S
        idx = order[-1]

        # push S in filtered predictions list
        # keep.append(P[idx])
        keep.append(idx.item())

        # remove S from P
        order = order[:-1]

        # sanity check
        if len(order) == 0:
            break

        # select coordinates of BBoxes according to
        # the indices in order
        xx1 = torch.index_select(x1,dim = 0, index = order)
        xx2 = torch.index_select(x2,dim = 0, index = order)
        yy1 = torch.index_select(y1,dim = 0, index = order)
        yy2 = torch.index_select(y2,dim = 0, index = order)

        # find the coordinates of the intersection boxes
        xx1 = torch.max(xx1, x1[idx])
        yy1 = torch.max(yy1, y1[idx])
        xx2 = torch.min(xx2, x2[idx])
        yy2 = torch.min(yy2, y2[idx])

        # find height and width of the intersection boxes
        w = xx2 - xx1
        h = yy2 - yy1

        # take max with 0.0 to avoid negative w and h
        # due to non-overlapping boxes
        w = torch.clamp(w, min=0.0)
        h = torch.clamp(h, min=0.0)

        # find the intersection area
        inter = w*h

        # find the areas of BBoxes according the indices in order
        rem_areas = torch.index_select(areas, dim = 0, index = order)

        # find the union of every prediction T in P
        # with the prediction S
        # Note that areas[idx] represents area of S
        union = (rem_areas - inter) + areas[idx]

        # find the IoU of every prediction in P with S
        IoU = inter / union

        # keep the boxes with IoU less than thresh_iou
        mask = IoU < thresh_iou
        order = order[mask]

    return keep

In [26]:
import torchvision
import cv2
from PIL import Image
import urllib.request
import os
from skimage import io
from skimage.transform import resize
from matplotlib import pyplot as plt
import numpy as np
import time
import pycuda.driver as cuda

class_names_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
                    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
                    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
                    'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
                    'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
                    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
                    'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
                    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
                    'scissors', 'teddy bear', 'hair drier', 'toothbrush']

# Preprocess the input image
def preprocess_image(img):
    result = torch.from_numpy(img).transpose(0,2).transpose(1,2)
    return np.array(result, dtype=np_target_dtype)



def tensorrt_run(image_array):
  results = predict(image_array)
  return results

def predict(batch): # result gets copied into output
    # transfer input data (from CPU) to device (GPU) asynchronously
    cuda.memcpy_htod_async(d_input, batch, stream)
    # execute model on the input data asynchronously
    context.execute_async_v2(bindings, stream.handle, None)
    # transfer predictions back from the device (GPU) to the host (CPU) asynchronously
    cuda.memcpy_dtoh_async(output_data , d_output, stream)
    # syncronize threads
    stream.synchronize()

    return output_data



def draw_boxes(tensorrt_results,image_array, video =False):
  """
  The raw output from yolo model contains information about bouding box coordinates and class probabilities.
  The first four indices corresponds to bounding box dimensions and the last dimension corresponds to class probabilities.
  This function will extract these information, retains bounding box with confidence score above 0.25 and perform an nms suppression to avoid detecting too many
  overlapping bounding boxes for a given object.The final output will be an image with bounding boxes drawn with clear labels to it.
  """

  # image_array = np.array(np.repeat(np.expand_dims(np.array(img), axis=0), BATCH_SIZE, axis=0)) * 255
  image_array = (image_array* 255).astype(np.uint8)

  save_dir = '/content/annotated_images'
  os.makedirs(save_dir, exist_ok=True)

  # tensorrt_results = torch.from_numpy(tensorrt_results[batch_idx])

  for batch_idx in range(tensorrt_results.shape[0]):
      # print(tensorrt_results[batch_idx].shape)
      predictions = torch.from_numpy(tensorrt_results[batch_idx]).permute(1, 0).unsqueeze(0)
      bounding_box_predictions = predictions[:, :, :4]
      class_probabilities = predictions[:, :, 4:]

      # Convert bounding box predictions to standard format
      x1 = bounding_box_predictions[:, :, 0] - bounding_box_predictions[:, :, 2] / 2
      y1 = bounding_box_predictions[:, :, 1] - bounding_box_predictions[:, :, 3] / 2
      x2 = bounding_box_predictions[:, :, 0] + bounding_box_predictions[:, :, 2] / 2
      y2 = bounding_box_predictions[:, :, 1] + bounding_box_predictions[:, :, 3] / 2
      bounding_boxes = torch.stack([x1, y1, x2, y2], dim=2)

      # Apply non-maximum suppression to bounding boxes
      confidence_scores = torch.max(class_probabilities, dim=2)[0]
      threshold = 0.25
      mask = confidence_scores >= threshold
      bounding_boxes = bounding_boxes[mask]
      class_probabilities = class_probabilities[mask]
      confidence_scores = confidence_scores[mask]

      # Apply non-maximum suppression to the bounding boxes
      nms_indices = nms_pytorch(bounding_boxes, confidence_scores, 0.45)
      selected_predictions = torch.cat([bounding_boxes[nms_indices], class_probabilities[nms_indices]], dim=1)

      # Extract bounding boxes and class labels for selected predictions
      selected_bounding_boxes = selected_predictions[:, :4]
      selected_class_probabilities = selected_predictions[:, 4:]
      selected_class_labels = torch.argmax(selected_class_probabilities, dim=1)

      # Annotate the image with bounding boxes and labels
      for i in range(len(selected_bounding_boxes)):
          x1, y1, x2, y2 = selected_bounding_boxes[i].int().tolist()
          label = class_names_list[selected_class_labels[i].item()]
          confidence = confidence_scores[nms_indices[i]].item()
          color = (255, 0, 0)  # Red color
          thickness = 2
          # image_array[batch_idx] = image_array[batch_idx]*255
          cv2.rectangle(image_array[batch_idx], (x1, y1), (x2, y2), color, thickness)
          cv2.putText(image_array[batch_idx], f'{label} {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, thickness)

      if not video:
        # Save the annotated image for the current batch
        save_path = os.path.join(save_dir, f'annotated_image_{batch_idx}.jpg')
        rgb_image = cv2.cvtColor(image_array[batch_idx], cv2.COLOR_BGR2RGB)
        cv2.imwrite(save_path, rgb_image)
  if video:
    return image_array

BATCH_SIZE = 1
# Download a sample image for inference
url='https://ultralytics.com/images/bus.jpg'
img = resize(io.imread(url), (640, 640))
input_batch = np.array(np.repeat(np.expand_dims(np.array(img, dtype=np.float32), axis=0), BATCH_SIZE, axis=0), dtype=np.float32)

preprocessed_images = np.array([preprocess_image(image) for image in input_batch])

# Create an execution context for inference using the built TensorRT engine
context = engine.create_execution_context()

# Allocate space for the output data, with the appropriate shape and data type
output_data = np.empty([BATCH_SIZE, 84, 8400], dtype = np_target_dtype)

# Allocate device memory for input and output
d_input = cuda.mem_alloc(1 * preprocessed_images.nbytes)
d_output = cuda.mem_alloc(1 * output_data.nbytes)

# Create a list of bindings, which are the addresses of the input and output buffers
bindings = [int(d_input), int(d_output)]

# Create a CUDA stream for asynchronous execution of TensorRT engine.
stream = cuda.Stream()
start_time = time.time()
tensorrt_results = tensorrt_run(preprocessed_images)
processing_time = time.time() - start_time
print("processing_time",processing_time )
draw_boxes(tensorrt_results,input_batch)




processing_time 0.0275270938873291


In [27]:
from google.colab import files

# Upload the video file
uploaded = files.upload()

# Get the path of the uploaded video file
video_path = next(iter(uploaded))

Saving video1 (online-video-cutter.com).mp4 to video1 (online-video-cutter.com) (1).mp4


In [28]:
from google.colab import files
import time
def process_video(onnx_path, input_video_path, output_video_path):

    """
    This function extracts the frames from uploaded video, detect the objects and draw bounding boxes with clear labels.
    The frames with the bounding box and detected labels are written back to an output video with fps equal
    corresponding to average processing time for each frome taken by the model
    """
    cap = cv2.VideoCapture(input_video_path)
    original_fps = cap.get(cv2.CAP_PROP_FPS)  # Get the FPS of the input video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, original_fps, (640, 640))  # Temporary FPS setting

    batch_frames = []
    BATCH_SIZE = 1 # Define batch size
    frame_count = 0
    total_processing_time = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_resized = cv2.resize(frame, (640, 640))
        batch_frames.append(frame_resized)

        if len(batch_frames) == BATCH_SIZE:
            batch_frames = np.array(batch_frames, dtype=np.float32) / 255.0
            batch_tensor = batch_frames.astype(np.float32)
            preprocessed_images = np.array([preprocess_image(image) for image in batch_tensor ])

            start_time = time.time()
            # pytorch_results = pytorch_run(model, batch_tensor)
            # onnx_results = onnx_run(onnx_path, batch_tensor)
            tensorrt_results = tensorrt_run(preprocessed_images)
            processing_time = time.time() - start_time
            # print("processing_time",processing_time )
            total_processing_time += processing_time
            frame_count += len(batch_frames)

            annotated_frames = draw_boxes(tensorrt_results, batch_frames, video=True)
            for annotated_frame in annotated_frames:
                fps_text = f'FPS: {frame_count / total_processing_time:.2f}'
                cv2.putText(annotated_frame, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                out.write(annotated_frame)

            batch_frames = []

    if batch_frames:  # Process remaining frames if any
        print("length of additional batch frames", len(batch_frames))
        last_frame = batch_frames[-1]
        while len(batch_frames) < BATCH_SIZE:
            batch_frames.append(last_frame)
        batch_frames = np.array(batch_frames, dtype=np.float32) / 255.0
        batch_tensor = batch_frames.astype(np.float32)

        start_time = time.time()
        tensorrt_results = tensorrt_run(preprocessed_images)
        processing_time = time.time() - start_time
        total_processing_time += processing_time
        frame_count += len(batch_frames)

        annotated_frames = draw_boxes(tensorrt_results, batch_frames, video=True)
        for annotated_frame in annotated_frames:
            fps_text = f'FPS: {frame_count / total_processing_time:.2f}'
            cv2.putText(annotated_frame, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            out.write(annotated_frame)

    cap.release()
    out.release()

    # Calculate the effective FPS
    effective_fps = frame_count / total_processing_time

    print("effective_fps",effective_fps )

    # Re-encode the video with the correct FPS
    temp_video_path = 'temp_output_video.mp4'
    cap = cv2.VideoCapture(output_video_path)
    out = cv2.VideoWriter(temp_video_path, fourcc, 60, (640, 640))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    cap.release()
    out.release()

    # Ensure the temporary file was created before renaming
    if os.path.exists(temp_video_path):
        os.remove(output_video_path)
        os.rename(temp_video_path, output_video_path)
    else:
        print("Error: temp_output_video.mp4 was not created successfully.")

# Set the ONNX model path
onnx_path = 'yolov8x.onnx'  # Path to your ONNX model

# Set the output video path
output_video_path = 'output_video.mp4'  # Path to save the output video

# Process the uploaded video
process_video(onnx_path, video_path, output_video_path)

# Download the output video
files.download(output_video_path)

effective_fps 63.3081439967135


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>